In [2]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

# 벡터DB : Chroma vs. Pinecone
- Chroma : 인메모리 vector DB, 로컬 vector DB
- Pinecone : 클라우드 vector DB
    (https://www.pinecone.io에서 api key 생성 -> .env에 추가(PINECONE_API_KEY등록)

# 0. 패키지 설치

In [3]:
%pip install -q pinecone langchain-pinecone --no-warn-script-location

Note: you may need to restart the kernel to use updated packages.


# 1. knowledge Base 구성을 위한 데이터 생성

In [6]:
%pip install langchain_community

   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 2.5/2.5 MB 29.1 MB/s  0:00:00
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 48.7 MB/s  0:00:00
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 2.1/2.1 MB 29.8 MB/s  0:00:00

   ---------- -----------------------------  3/11 [greenlet]
   ---------- -----------------------------  3/11 [greenlet]
  Attempting uninstall: async-timeout
   ---------- -----------------------------  3/11 [greenlet]
    Found existing installation: async-timeout 5.0.1
   ---------- -----------------------------  3/11 [greenlet]
    Uninstalling async-timeout-5.0.1:
   ---------- -----------------------------  3/11 [greenlet]
      Successfully uninstalled async-timeout-5.0.1
   ---------- -----------------------------  3/11 [greenlet]
   ---------------------

In [7]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
loader = Docx2txtLoader('data/소득세법(법률)(제21065호)(20260102).docx')
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, 
    chunk_overlap=200,
    # separators=["\n\n", "\n", " ", ""]
)
document_list = loader.load_and_split(text_splitter=text_splitter)
len(document_list)

ValueError: File path data/소득세법(법률)(제21065호)(20260102).docx is not a valid file or url

In [3]:
# embedding : upstage의 solar-embedding-1-large-passage
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings
load_dotenv()
embedding = UpstageEmbeddings(model="solar-embedding-1-large-passage")

In [4]:
len(embedding.embed_query("소득세법"))

4096

In [8]:
%%time
# pinecone vector database
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
import os
pc = Pinecone(
    api_key=os.getenv("PINECONE_API_KEY")
)
# 데이터를 처음 업로드할 때
index_name = "tax-index-upstage"
# database = PineconeVectorStore.from_documents(
#     documents=document_list,
#     embedding=embedding,
#     index_name=index_name
# )
# 업로드시 경고가 안보이려면 아나콘다 프롬프트 llm환경에서 conda install -c conda-forge ipywidgets

# 업로드한 벡터db를 가져올 때
database = PineconeVectorStore(
    embedding=embedding,# 질문을 임베딩하여 유사도 검색
    index_name=index_name
)

CPU times: total: 0 ns
Wall time: 2.99 ms


# 2. 답변 생성을 위한 Retrieval

In [9]:
query = "연봉이 5천만원인 직장인의 소득세는 얼마인가요?"
retrieved_docs = database.similarity_search(query, k=3) # 기본k값:4

In [22]:
# retrieved_docs[2].page_content
retrieved_doc = "\n\n---\n\n".join([doc.page_content for doc in retrieved_docs])

In [10]:
retriever = database.as_retriever(
    search_kwargs={"k":3}
)
retrived_docs = retriever.invoke(query)
retrieved_doc = "\n\n---\n\n".join([doc.page_content for doc in retrieved_docs])

# 3. 답변 생성

In [24]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model = "gpt-4.1-nano")

In [12]:
# upstage에서 받은 20$로 llm을 사용하고 싶다면
from langchain_upstage import ChatUpstage
llm = ChatUpstage(
    model = "solar-pro2",
    reasoning_effort="high" #느리지만 더 깊게 추론함 (low, medium)
)

In [13]:
prompt = f"""[identity]
- 당신은 최고의 한국 소득세법 전문가입니다
- [context]를 참고해서 사용자의 질문에 답변해 주세요.
- [context]는 다음과 같아요
{retrieved_doc}
- 질문 : {query}"""

In [14]:
ai_message = llm.invoke(prompt)

In [15]:
print(ai_message.content)

연봉 5천만원인 직장인의 소득세를 계산하는 과정은 다음과 같습니다. 단, 추가 공제(자녀, 연금계좌 등)는 고려하지 않습니다.

---

### **1. 근로소득공제 계산**
근로소득공제는 총급여액에 따라 다음과 같이 적용됩니다(제47조).
- **5천만원**에 대한 근로소득공제:
  - 500만원 이하: 70% 공제 (500만원 × 70% = 350만원)
  - 500만원 초과 ~ 1,500만원: 40% 공제 (1,000만원 × 40% = 400만원)
  - 1,500만원 초과 ~ 4,500만원: 15% 공제 (3,000만원 × 15% = 450만원)
  - 4,500만원 초과 ~ 5,000만원: 5% 공제 (500만원 × 5% = 25만원)
  - **총 근로소득공제**: 350 + 400 + 450 + 25 = **1,225만원**

---

### **2. 과세표준 산정**
- **총급여액**: 5,000만원  
- **근로소득공제 후 과세표준**: 5,000만원 - 1,225만원 = **3,775만원**

---

### **3. 종합소득산출세액 계산**
2023년 종합소득세 세율 및 누진공제액을 적용합니다(제55조).
- **3,775만원**은 **1,400만원 초과 ~ 5,000만원 이하** 구간에 해당합니다.
  - **세율**: 15%  
  - **누진공제액**: 120만원  
  - **산출세액**: (3,775만원 × 15%) - 120만원 = **566.25만원 - 120만원 = 446.25만원**

---

### **4. 근로소득세액공제 적용**
총급여액 5,000만원은 **3,300만원 초과 ~ 7,000만원 이하** 구간에 해당합니다(제59조).
- **공제액**:  
  \[
  74만원 - \left( (5,000만원 - 3,300만원) × \frac{8}{1,000} \right) = 74만원 - 13.6만원 = 60.4만원
  \]
  - 단, **60.4만원 < 66만원**이므로 **66만원** 공제 적용.
- **최종 

# 4. langchain 답변 생성
- ch9.07_LangChain과 vectorDatabase을 활용한 RAG구현(UpstageEmbedding) 참조

In [16]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from dotenv import load_dotenv

# 1. LLM과 임베딩 초기화
load_dotenv()
# llm = ChatOpenAI(model = "gpt-4.1-mini")
from langchain_upstage import ChatUpstage
llm = ChatUpstage(model="solar-pro2")

embedding = UpstageEmbeddings(model="solar-embedding-1-large-passage")

# 2. # 업로드한 벡터db를 가져올 때
vectorstore = PineconeVectorStore(
    embedding=embedding,# 질문을 임베딩하여 유사도 검색
    index_name=index_name
)
# 3. Retriever 생성
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":4})
# 4. 프롬프트 템플릿
template = f"""당신은 최고의 한국 소득세 전문가입니다.
다음 문맥을 참고하여 질문에 답하세요.
답을 모르면 모른다고 답하세요.
최대 3문장으로 간결하게 답변하세요.
질문 : {{query}}
문맥 : {{context}}
답변 : """
prompt = ChatPromptTemplate.from_template(template)
# 5. 검색된 document를 텍스트로 변환하는 함수
def format_documents(documents):
    return  "\n\n---\n\n".join([doc.page_content for doc in documents])

In [17]:
# 6. RAG 체인 구성 (LCEL 방식)
from langchain_core.runnables import RunnablePassthrough # {"query":"~"} => "~"
rag_chain = (
    {
        "context":retriever | format_documents,
        "query":RunnablePassthrough() # 질문 그대로 전달
    }
    | prompt # prompt에 context와 query 변수 주입
    | llm    
    | StrOutputParser()
)
# 7. 실행
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"
rag_chain.invoke(query)

'제공된 문맥에는 연봉 5천만원에 대한 구체적인 소득세 계산 방법이 명시되어 있지 않습니다. 따라서 정확한 세액을 계산할 수 없으며, "모름"으로 답변드립니다. 단, 근로소득 공제 및 세액 공제 규정을 참고하여 국세청의 세금 계산기 등을 활용하면 예상 세액을 확인할 수 있습니다.'